In [1]:
!pip install torch flask nest_asyncio transformers ipython pyngrok scikit-learn

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
!pip uninstall pyngrok
!pip install pyngrok


  Using cached pyngrok-7.2.1-py3-none-any.whl.metadata (8.3 kB)


In [3]:
import torch
from flask import Flask, request, jsonify, send_from_directory
import nest_asyncio
from IPython.display import display, HTML
import os
from transformers import AutoTokenizer, AutoModelForCausalLM


app = Flask(__name__)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Advanced Storytelling Bot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f3f4f6;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
        }
        .container {
            text-align: left;
            width: 90%;
            max-width: 800px;
            background-color: white;
            padding: 30px;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
        }
        .input-group {
            margin-bottom: 20px;
        }
        .input-group label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
        input[type="text"], textarea, select {
            width: 100%;
            padding: 8px;
            margin: 5px 0;
            border: 1px solid #ddd;
            border-radius: 4px;
            box-sizing: border-box;
        }
        textarea {
            height: 100px;
            resize: vertical;
        }
        #storyOutput {
            margin-top: 20px;
            padding: 20px;
            background-color: #f8f9fa;
            border-radius: 5px;
            text-align: left;
            white-space: pre-wrap;
            min-height: 200px;
            border: 1px solid #ddd;
            line-height: 1.6;
        }
        button {
            padding: 10px 20px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }
        button:disabled {
            background-color: #cccccc;
        }
        #loading {
            display: none;
            margin-top: 10px;
            color: #666;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Advanced Storytelling Bot</h1>

        <div class="input-group">
            <label>Story Title:</label>
            <input type="text" id="storyTitle" placeholder="Enter a title for your story">
        </div>

        <div class="input-group">
            <label>Genre:</label>
            <select id="genre">
                <option value="general">General</option>
                <option value="comedy">Comedy</option>
                <option value="mystery">Mystery</option>
                <option value="horror">Horror</option>
                <option value="scifi">Sci-Fi</option>
                <option value="fantasy">Fantasy</option>
                <option value="romance">Romance</option>
            </select>
        </div>

        <div class="input-group">
            <label>Characters:</label>
            <textarea id="characters" placeholder="List characters and brief descriptions"></textarea>
        </div>

        <div class="input-group">
            <label>Story Setting:</label>
            <input type="text" id="setting" placeholder="Describe where and when the story takes place">
        </div>

        <div class="input-group">
            <label>Additional Plot Elements:</label>
            <textarea id="plotElements" placeholder="Add specific plot elements, twists, or events"></textarea>
        </div>

        <button onclick="generateStory()" id="generateBtn">Generate Story</button>
        <div id="loading">Crafting your unique story... This may take a minute...</div>
        <div id="storyOutput"></div>
    </div>

    <script>
        async function generateStory() {
            const generateBtn = document.getElementById('generateBtn');
            const loading = document.getElementById('loading');
            const storyOutput = document.getElementById('storyOutput');

            const storyData = {
                title: document.getElementById('storyTitle').value,
                genre: document.getElementById('genre').value,
                characters: document.getElementById('characters').value,
                setting: document.getElementById('setting').value,
                plotElements: document.getElementById('plotElements').value
            };

            // Disable inputs and show loading
            generateBtn.disabled = true;
            loading.style.display = 'block';
            storyOutput.innerText = '';

            try {
                const response = await fetch('/generate_story', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(storyData)
                });

                if (!response.ok) {
                    throw new Error('Failed to generate story');
                }

                const data = await response.json();
                storyOutput.innerText = data.story;
            } catch (error) {
                storyOutput.innerText = 'Error generating story. Please try again.';
                console.error('Error:', error);
            } finally {
                generateBtn.disabled = false;
                loading.style.display = 'none';
            }
        }
    </script>
</body>
</html>
"""


with open('index.html', 'w') as f:
    f.write(html_content)

@app.route('/')
def serve_index():
    return send_from_directory('.', 'index.html')

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.get_json()


        prompt = construct_story_prompt(data)


        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        outputs = model.generate(
            inputs['input_ids'],
            max_length=1000,
            num_return_sequences=1,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            repetition_penalty=1.2
        )

        story = tokenizer.decode(outputs[0], skip_special_tokens=True)
        story = story.replace(prompt, "").strip()


        if data.get('title'):
            story = f"{data['title']}\n\n{story}"

        return jsonify({"story": story})

    except Exception as e:
        return jsonify({"error": str(e)}), 500





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
def construct_story_prompt(data):
    """Constructs a dynamic prompt based on user input"""
    prompt_parts = []


    title = data.get('title', 'Untitled')
    genre = data.get('genre', 'general')
    prompt_parts.append(f"Title: {title}")


    genre_instructions = {
        'comedy': 'Write a humorous story with witty dialogue and comedic situations.',
        'mystery': 'Create a suspenseful story with clues, twists, and a satisfying revelation.',
        'horror': 'Craft a chilling tale with growing tension and atmospheric dread.',
        'scifi': 'Develop a story with futuristic elements and scientific concepts.',
        'fantasy': 'Write a magical tale with fantastical elements and world-building.',
        'romance': 'Create a story focused on relationship development and emotional connection.',
        'general': 'Write an engaging story with well-developed characters and plot.'
    }

    prompt_parts.append(f"Genre: {genre}. {genre_instructions.get(genre.lower(), genre_instructions['general'])}")


    if data.get('setting'):
        prompt_parts.append(f"Setting: The story unfolds in {data['setting']}.")

    if data.get('characters'):
        prompt_parts.append(f"Characters: {data['characters']}")

    if data.get('plotElements'):
        prompt_parts.append(f"Plot Elements: Incorporate the following elements: {data['plotElements']}")

    prompt_parts.append(f"Create a compelling {genre} story that naturally incorporates all these elements. Focus on character development, engaging dialogue, and a satisfying narrative arc.")

    full_prompt = "\n".join(prompt_parts) + "\n\nBegin the story:"
    return full_prompt


In [5]:
def construct_story_prompt(data):
    """Constructs a detailed prompt for story generation"""

    prompt_template = f"""
You are tasked to create a new story based on the following elements:

Title: {data.get('title', 'Untitled')}

Character Names: {data.get('characters')}

Setting: {data.get('setting')}

Plot: {data.get('plotElements')}

Story Guidelines:
- Maintain a {data.get('genre', 'comedic')} tone throughout.
- Include vivid descriptions and engaging dialogue.
- Develop character relationships naturally, showing the growth of the characters.
- The plot should build towards a satisfying, funny resolution.
- Keep the narrative focused and coherent, avoiding unnecessary details.

Start the story based on these instructions:
"""
    return prompt_template.strip()


In [6]:

nest_asyncio.apply()


from pyngrok import ngrok, conf
conf.get_default().auth_token = "2iT0ijEpwf62UnrkvHmP1aAvdJC_ex3C4AFs2djuBfZPDPQa"

ngrok.kill()

public_url = ngrok.connect(addr="5000", proto="http")
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run(port=5000, debug=False)

Public URL: NgrokTunnel: "https://c2cb-34-82-34-100.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
